#### Import Required Libraries

In [21]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
import xgboost as xgb
import pickle
from sklearn.metrics import accuracy_score, classification_report, f1_score, confusion_matrix
import optuna

In [22]:
train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")

In [23]:
X_train = train.drop("Exited", axis=1)
y_train = train["Exited"]

X_test = test.drop("Exited", axis=1)
y_test = test["Exited"]

In [24]:
def engg_features(X):
    X["BalanceSalaryRatio"] = X["Balance"] / X["EstimatedSalary"]
    X["TenureByAge"] = X["Tenure"] / X["Age"]
    X["CreditScoreGivenAge"] = X["CreditScore"] / X["Age"]
    X["HasBalance"] = np.where(X["Balance"] > 0, 1, 0)
    X["ActiveByAge"] = X["IsActiveMember"] * X["Age"]
    X['AgeCategory'] = pd.cut(X['Age'], bins=[0, 35, 55, np.inf], labels=['Young', 'MiddleAge', 'Senior'])
    return X


In [25]:
preprocessing_pipeline = pickle.load(open("../models/pipeline.pkl", "rb"))

In [26]:
models = {
    "logistic regression": LogisticRegression(),
    "random forest": RandomForestClassifier(),
    "gradient boosting": HistGradientBoostingClassifier(),
    "svc": SVC(),
    "xgboost": xgb.XGBClassifier()
}

results = {}

for name, model in models.items():
    model_pipeline = Pipeline([
        ("preprocessing_pipeline", preprocessing_pipeline),
        ("model", model)
    ])

    model_pipeline.fit(X_train, y_train)
    y_pred = model_pipeline.predict(X_test)

    f1 = f1_score(y_test, y_pred)
    results[name] = f1

results
    

d:\AI-ML\Projects\Customer-Churn-Prediction\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


{'logistic regression': 0.35887850467289717,
 'random forest': 0.5740458015267176,
 'gradient boosting': 0.5973254086181278,
 'svc': 0.0,
 'xgboost': 0.5611510791366906}

#### Hyperparameter Tuning Using Optuna

In [7]:
def objective(trial):
    classifier_name = trial.suggest_categorical("classifier", ["LogisticRegression", "RandomForest", "GradientBoosting", "SVC", "XGBoost"])

    # xgb_weight = np.sum(y_train == 0) / np.sum(y_train == 1)


    if classifier_name == "LogisticRegression":
        lr_params = {
            "C": trial.suggest_float("C", 0.0001, 10, log=True),
            "solver": trial.suggest_categorical("solver", ["liblinear", "lbfgs"]),
            "class_weight": "balanced",
            "max_iter": 1000,
            "random_state": 42
        }

        model = LogisticRegression(**lr_params)
    
    elif classifier_name == "RandomForest":
        bootstrap = trial.suggest_categorical("bootstrap", [True, False])

        rf_params = {
            "n_estimators": trial.suggest_int("n_estimators", 50, 300),
            "max_depth": trial.suggest_int("max_depth", 3, 20),
            "min_samples_split": trial.suggest_int("min_samples_split", 2, 10),
            "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 10),
            "bootstrap": bootstrap,
            "class_weight": "balanced_subsample" if bootstrap else "balanced", 
            "random_state": 42
        }

        model = RandomForestClassifier(**rf_params)

    elif classifier_name == "GradientBoosting":
        # gb_params = {
        #     "n_estimators": trial.suggest_int("n_estimators", 50, 100),
        #     "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        #     "max_depth": trial.suggest_int("max_depth", 3, 20),
        #     "min_samples_split": trial.suggest_int("min_samples_split", 2, 10),
        #     "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 10),
        #     "random_state": 42
        # }

        # model = GradientBoostingClassifier(**gb_params)

        hgb_params = {
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
            "max_iter": trial.suggest_int("max_iter", 50, 300),
            "max_depth": trial.suggest_int("max_depth", 3, 20),
            "l2_regularization": trial.suggest_float("l2_regularization", 0, 10),
            "class_weight": "balanced",
            "random_state": 42
        }

        model = HistGradientBoostingClassifier(**hgb_params)
    
    elif classifier_name == "SVC":        
        svc_params = {
            "C": trial.suggest_float("C", 0.1, 100, log=True),
            "kernel": trial.suggest_categorical("kernel", ["linear", "rbf", "poly", "sigmoid"]), 
            "gamma": trial.suggest_categorical("gamma", ["scale", "auto"]),
            "class_weight": "balanced",
            "random_state": 42
        }
        
        model = SVC(**svc_params)

    elif classifier_name == "XGBoost":
        xgb_params = {
        'learning_rate': trial.suggest_float("xgb_learning_rate", 0.01, 0.3),
        'n_estimators': trial.suggest_int("xgb_n_estimators", 50, 300),
        'max_depth': trial.suggest_int("xgb_max_depth", 3, 10),
        'subsample': trial.suggest_float("xgb_subsample", 0.5, 1.0),
        'scale_pos_weight': trial.suggest_float("xgb_scale_pos_weight", 2.0, 6.0),
        'random_state': 42,
        'verbosity': 0
        }
        
        model = xgb.XGBClassifier(**xgb_params)

    
    pipeline = Pipeline([
        ("preprocessing", preprocessing_pipeline), 
        ("model", model)
    ])

    scores = cross_val_score(pipeline, X_train, y_train, cv=3, scoring="f1", n_jobs=-1)
    return scores.mean()

In [8]:
print("-" * 50)

study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler())
study.optimize(objective, n_trials=100)

print(f"Best Accuracy: {study.best_value:.4f}")
print(f"Best Params: {study.best_params}")

[I 2025-12-11 15:21:09,997] A new study created in memory with name: no-name-ab6ac5ff-6cf4-4bf1-95d9-680a79cd114a


--------------------------------------------------


[I 2025-12-11 15:21:15,763] Trial 0 finished with value: 0.508250218423068 and parameters: {'classifier': 'SVC', 'C': 0.1034798667608419, 'kernel': 'linear', 'gamma': 'auto'}. Best is trial 0 with value: 0.508250218423068.
[I 2025-12-11 15:21:18,459] Trial 1 finished with value: 0.5979618858931963 and parameters: {'classifier': 'XGBoost', 'xgb_learning_rate': 0.06603484354415998, 'xgb_n_estimators': 74, 'xgb_max_depth': 5, 'xgb_subsample': 0.8400259237962702, 'xgb_scale_pos_weight': 4.538679640708576}. Best is trial 1 with value: 0.5979618858931963.
[I 2025-12-11 15:21:20,790] Trial 2 finished with value: 0.5026668877622429 and parameters: {'classifier': 'LogisticRegression', 'C': 0.18087286685625342, 'solver': 'lbfgs'}. Best is trial 1 with value: 0.5979618858931963.
[I 2025-12-11 15:21:23,718] Trial 3 finished with value: 0.6037509987037626 and parameters: {'classifier': 'RandomForest', 'bootstrap': False, 'n_estimators': 257, 'max_depth': 20, 'min_samples_split': 9, 'min_samples_lea

Best Accuracy: 0.6171
Best Params: {'classifier': 'RandomForest', 'bootstrap': True, 'n_estimators': 52, 'max_depth': 20, 'min_samples_split': 3, 'min_samples_leaf': 8}


In [9]:
study.trials_dataframe()["params_classifier"].value_counts()

params_classifier
RandomForest          59
GradientBoosting      19
SVC                    8
LogisticRegression     7
XGBoost                7
Name: count, dtype: int64

In [10]:
study.trials_dataframe().groupby("params_classifier")["value"].mean().sort_values(ascending=False)

params_classifier
RandomForest          0.609887
GradientBoosting      0.606161
XGBoost               0.580448
SVC                   0.504890
LogisticRegression    0.487418
Name: value, dtype: float64

In [27]:
if study.best_params["classifier"] == "LogisticRegression":
    model = LogisticRegression(**dict(list(study.best_params.items())[1:]))
elif study.best_params["classifier"] == "RandomForest":
    model = RandomForestClassifier(**dict(list(study.best_params.items())[1:]))
elif study.best_params["classifier"] == "GradientBoosting":
    model = HistGradientBoostingClassifier(**dict(list(study.best_params.items())[1:]))
elif study.best_params["classifier"] == "SVC":
    model = SVC(**dict(list(study.best_params.items())[1:]))
elif study.best_params["classifier"] == "XGBoost": 
    model = xgb.XGBClassifier(**dict(list(study.best_params.items())[1:]))

pipeline = Pipeline([
        ("preprocessing", preprocessing_pipeline), 
        ("model", model)
    ])

pipeline.fit(X_train, y_train)

,steps,"[('preprocessing', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,steps,"[('feature_engg', ...), ('preprocessor', ...)]"
,transform_input,None
,memory,None
,verbose,False
,func,<function eng...0015156382160>
,inverse_func,None
,validate,False


In [28]:
y_prob = pipeline.predict_proba(X_test)[:,1]

threshold = 0.35
y_pred = (y_prob >= threshold).astype(int)

In [29]:
print(f1_score(y_test, y_pred))

0.6134020618556701


In [30]:
print(confusion_matrix(y_test, y_pred))

[[1462  131]
 [ 169  238]]


In [31]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.90      0.92      0.91      1593
           1       0.64      0.58      0.61       407

    accuracy                           0.85      2000
   macro avg       0.77      0.75      0.76      2000
weighted avg       0.85      0.85      0.85      2000



In [32]:
pickle.dump(pipeline, open("../models/final_model_pipeline.pkl","wb"))